In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
air_joined = pd.read_parquet('airtable_joined.parquet')

#### Run all operators

In [3]:
analysis_date = dt.date(2022, 8, 3) ##wednesday, new tables

In [4]:
day = str(analysis_date.day).zfill(2)
month = str(analysis_date.month).zfill(2)

In [5]:
fs_list = fs.ls(f'{shared_utils.rt_utils.GCS_FILE_PATH}rt_trips/')

In [6]:
## now finds ran operators on specific analysis date
ran_operators = [int(path.split('rt_trips/')[1].split('_')[0])
                 for path in fs_list
                 if path.split('rt_trips/')[1] and path.split('rt_trips/')[1].split('_')[1] == month and path.split('rt_trips/')[1].split('_')[2][:2] == day]

In [7]:
len(ran_operators)

0

In [8]:
ran_operators

[]

In [9]:
pbar = tqdm()

0it [00:00, ?it/s]

In [11]:
# analysis_date = dt.date(2022, 2, 8) ##tuesday for sacRT

In [12]:
not_ran_operators = []
# for agency in air_joined.calitp_itp_id.unique():
for agency in [208]:
    if agency in ran_operators:
        print(f'already ran: {agency}')
        continue
    ## https://docs.google.com/spreadsheets/d/16tcL3fPdYkrNajDNneSR1b5ImhFOCJ0gWFnEaXyg16A/edit#gid=0
    print(f'calculating for agency: {agency}...')
    try:
        rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)
        rt_day.export_views_gcs()
        print(f'complete for agency: {agency}')
    except Exception as e:
        print(f'rt failed for agency {agency}')
        not_ran_operators += [agency]
        print(e)

calculating for agency: 208...
getting trips...


/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:39: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:39: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a 

vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
vehicle positions gdf must not be empty
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data


/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:39: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:39: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a 

complete for agency: 208


In [15]:
ran_operators += [273]

In [16]:
import yaml
import pyaml

In [17]:
airtable_organizations = (
    tbl.airtable.california_transit_organizations()
    >> select(_.itp_id, _.name, _.caltrans_district,
              _.website, _.ntp_id, _.drmt_organization_name)
    # >> filter(_.itp_id == itp_id)
    >> collect()
)

In [18]:
airtable_organizations = airtable_organizations.dropna(subset=['itp_id']) >> mutate(itp_id = _.itp_id.astype('int64'))

In [19]:
airtable_organizations = airtable_organizations >> filter(_.itp_id.isin(ran_operators)) >> arrange(_.caltrans_district)

In [20]:
airtable_organizations.head(3)

,itp_id,name,caltrans_district,website,ntp_id,drmt_organization_name
679,135,Humboldt Transit Authority,01 - Eureka,None,['reccKueWkvBCZoLQs'],['recu9h9nAElLpYoYO']
588,259,Redding Area Bus Authority,02 - Redding,None,['recoXZjKEOPldH8QJ'],None
655,221,Nevada County,03 - Marysville,https://www.mynevadacounty.com/,['recx7U6zAWSOGpcyR'],None


In [21]:
with open('../portfolio/sites/rt.yml') as rt_site:
    analyses_data = yaml.load(rt_site, yaml.Loader)

In [22]:
exclude_ids = [312, 315, 246] ##rail only systems

In [23]:
chapters_list = []
for district in airtable_organizations.caltrans_district.unique():
    if type(district) == type(None):
        continue
    chapter_dict = {}
    filtered = (airtable_organizations
                >> filter(_.caltrans_district == district)
                >> distinct(_.itp_id, _keep_all=True)
                >> filter(-_.itp_id.isin(exclude_ids))
               )
    chapter_dict['caption'] = f'District {district}'
    chapter_dict['params'] = {'district': district}
    chapter_dict['sections'] = [{'itp_id': itp_id} for itp_id in filtered.itp_id.to_list()]
    chapters_list += [chapter_dict]

In [24]:
parts_list = [{'chapters': chapters_list}]

In [25]:
# parts_list

In [26]:
# analyses_data['parts']

In [27]:
analyses_data['parts'] = parts_list

In [28]:
output = pyaml.dump(analyses_data)

In [29]:
with open('../portfolio/sites/test_rt.yml', 'w') as rt_site:
    rt_site.write(output)

In [67]:
# with open('../portfolio/sites/rt.yml', 'w') as rt_site:
#     rt_site.write(output)

### Ongoing issues
* 358 Union City too many trips without shape?

#### Parameters
* 159 1+ maps don't render
* 167 map_from_metrics fails